In [1]:
# Requires brainvisa

In [1]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [2]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [3]:
%matplotlib qt5
a = anatomist.Anatomist()

Starting Anatomist.....
global modules: /neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/anatomist-5.2/python_plugins
home   modules: /home/rm283713/.anatomist/python_plugins
config file : /home/rm283713/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module gltf_io
loading module palettecontrols
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = ""
path_file = "/neurospin/dico/rmenasria/Runs/01_essai/Program/2023_jlaval_STSbabies/contrastive/notebooks/racim/csv/prematurity_AUC_by_region_1205.csv"


In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
file_to_display = f"{path_summary}/{path_file}"

In [7]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [8]:
param = "AUC_27_32"

In [9]:
df = pd.read_csv(file_to_display)[["region", param]]
df.head()

,region,AUC_27_32
0,CINGULATE_left,0.703681
1,CINGULATE_right,0.718578
2,FCLp-subsc-FCLa-INSULA_left,0.789564
3,FCLp-subsc-FCLa-INSULA_right,0.871490
4,FCMpost-SpC_left,0.637360


In [10]:

res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [11]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [12]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [13]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [14]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [15]:
res["region"].unique()

array(['FCLp-subsc-FCLa-INSULA_left', 'FCLp-subsc-FCLa-INSULA_right',
       'FCMpost-SpC_left', 'FCMpost-SpC_right', 'FColl-SRh_left',
       'FColl-SRh_right', 'FIP_left', 'FIP_right', 'FPO-SCu-ScCal_left',
       'FPO-SCu-ScCal_right', 'Lobule_parietal_sup_left',
       'Lobule_parietal_sup_right', 'OCCIPITAL_left', 'OCCIPITAL_right',
       'SC-SPeC_left', 'SC-SPeC_right', 'SC-SPoC_left', 'SC-SPoC_right',
       'SC-sylv_left', 'SC-sylv_right', 'SFinf-BROCA-SPeCinf_left',
       'SFinf-BROCA-SPeCinf_right', 'SFint-FCMant_left',
       'SFint-FCMant_right', 'SFint-SR_left', 'SFint-SR_right',
       'SFinter-SFsup_left', 'SFinter-SFsup_right',
       'SFmarginal-SFinfant_left', 'SFmarginal-SFinfant_right',
       'SFmedian-SFpoltr-SFsup_left', 'SFmedian-SFpoltr-SFsup_right',
       'SOr-SOlf_left', 'SOr-SOlf_right', 'SOr_left', 'SOr_right',
       'SPeC_left', 'SPeC_right', 'SPoC_left', 'SPoC_right',
       'STi-SOTlat_left', 'STi-SOTlat_right', 'STi-STs-STpol_left',
       'STi-STs-

In [17]:
# res["region"].tolist()

In [16]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "SOr": "S.Or.",
    "SOlf": "S.Olf.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [17]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res.head()

,region,AUC_27_32,side
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,0.789564,left
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.871490,right
4,F.C.M.post.-S.p.C._left,0.637360,left
5,F.C.M.post.-S.p.C._right,0.660273,right
6,F.Coll.-S.Rh._left,0.755726,left


In [18]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,AUC_27_32,side,sulcus
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,0.789564,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.871490,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
4,F.C.M.post.-S.p.C._left,0.637360,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"
5,F.C.M.post.-S.p.C._right,0.660273,right,"[F.C.M.post._right, S.C.LPC._right, S.p.C._right]"
6,F.Coll.-S.Rh._left,0.755726,left,"[F.Coll._left, S.Rh._left]"


In [19]:
res = res.sort_values(by=param, ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,AUC_27_32,side,sulcus
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.i.ant._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.i.post._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.pol._right
49,S.T.s._right,0.849937,right,S.T.s._right
46,S.T.i.-S.T.s.-S.T.pol._left,0.815399,left,S.T.i.ant._left
46,S.T.i.-S.T.s.-S.T.pol._left,0.815399,left,S.T.i.post._left
46,S.T.i.-S.T.s.-S.T.pol._left,0.815399,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,0.815399,left,S.T.pol._left
48,S.T.s._left,0.809193,left,S.T.s._left


In [20]:
for _, row in res.iterrows():
    print(row.sulcus)

S.T.i.ant._right
S.T.i.post._right
S.T.s._right
S.T.pol._right
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
S.T.s._right
S.F.int._left
S.R.inf._left
S.R.sup._left
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
S.T.s._left
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._left
S.C._left
S.C.sylvian._left
S.Po.C.sup._left
F.I.P.Po.C.inf._left
F.C.L.r.retroC.tr._left
S.Po.C.sup._right
F.I.P.Po.C.inf._right
F.C.L.r.retroC.tr._right
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
F.C.M.ant._left
S.F.int._left
F.Coll._right
S.Rh._right
F.Coll._left
S.Rh._left
S.F.int._right
S.R.inf._right
S.R.sup._right
S.Po.C.sup._left
F.I.P.Po.C.inf._left
F.C.L.r.retroC.tr._left
F.C.M.ant._right
S.F.int._right
F.Cal.ant.-Sc.Cal._right
S.O.p._

In [21]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,AUC_27_32,side,sulcus
33,S.F.marginal-S.F.inf.ant._right,0.664022,right,S.F.orbitaire._right


In [22]:
res

,region,AUC_27_32,side,sulcus
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.i.ant._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.i.post._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,0.873431,right,S.T.pol._right
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.871490,right,F.C.L.p._right
...,...,...,...,...
51,S.T.s.br._right,0.575739,right,F.I.P.r.int.1_right
51,S.T.s.br._right,0.575739,right,F.I.P.r.int.2_right
51,S.T.s.br._right,0.575739,right,S.T.s.ter.asc.ant._right
51,S.T.s.br._right,0.575739,right,S.T.s.ter.asc.post._right


In [23]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row[param])
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=0.4, maxVal=1,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [24]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 8
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


: 